# Extremwerte
_CAS Data Science FHNW, 2023, S. Billeter_

**Dieses Skript dient in einer Live-Demonstration zur Erklärung und muss vor einer Anwendung zwingend angepasst werden**

In [ ]:
# Libraries installieren wenn nötig
#!pip install statsmodels
#!pip install pyextremes

In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import statsmodels
from statsmodels.tsa.seasonal import seasonal_decompose
import scipy

### Vorbereitung der Extremwertanalyse

Daten vorbereiten

In [ ]:
# Gleiche Vorbereitung wie im ersten Skript
df_Luzern=pd.read_csv('Luzern-TS.csv',sep=';')
df_Luzern['Date']=pd.to_datetime(df_Luzern['Year'].astype(str).str.cat(df_Luzern['Month'].astype(str),sep='-'))
Luzern_Temp=df_Luzern[['Date','Temperature']]
Luzern_Temp=Luzern_Temp.set_index('Date')
Luzern_Temp=Luzern_Temp[Luzern_Temp.index>='1900-01-01']
Luzern_Temp_Komp=seasonal_decompose(Luzern_Temp, model='additive')
fig=Luzern_Temp_Komp.plot()
fig.set_size_inches(12,6)
plt.show()

In [ ]:
# Residuen über drei Referenzperioden
Luzern_Temp_Resid=pd.Series.to_frame(Luzern_Temp_Komp.resid)
# Luzern_Temp_Resid[Luzern_Temp_Resid.resid.notnull()]
Luzern_Temp_Resid=Luzern_Temp_Resid[Luzern_Temp_Resid.index>='1920-01-01']
Luzern_Temp_Resid=Luzern_Temp_Resid[Luzern_Temp_Resid.index<='2019-12-31']
Luzern_Temp_Resid

In [ ]:
Residuen=pd.to_numeric(Luzern_Temp_Resid.resid).values
Residuen

**Extremwerte**: Passt die Normalverteilung?

In [ ]:
# Zeitliche Verteilung vernachlässigt
fig,ax=plt.subplots()
fig.set_size_inches(8,6)
ax.plot(Residuen,'o')
plt.show()

In [ ]:
# Wie viele wie grosse Werte gibt es?
fig,ax=plt.subplots()
fig.set_size_inches(8,6)
ax.hist(Residuen,bins=20, density=True, label='Residuen')
plt.show()

In [ ]:
# Histogramm mit Normalverteilung
fig,ax=plt.subplots()
fig.set_size_inches(8,6)
plt.hist(Residuen,bins=20, density=True, label='Residuen')

from scipy.stats import norm

# Wahrscheinlichkeitsdichte normalverteilt
mu, sig = norm.fit(Residuen) 
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, sig)
  
plt.plot(x, p, 'k', linewidth=2, label='Normalverteilung')
# plt.tight_layout()
plt.legend()
plt.title('Residuen Temperatur')
plt.show()
print(mu)
print(sig)

In [ ]:
# Kumulierte Wahrscheinlichkeit, Normalverteilung
# Emipirische Verteilung
from statsmodels.distributions import ECDF
res_cdf=ECDF(Residuen)

fig,ax=plt.subplots()
fig.set_size_inches(8,6)

plt.plot(x,res_cdf(x),label='Residuen')
plt.plot(x,norm.cdf(x,mu,sig),label='Normalverteilung')
plt.legend()
plt.title('Residuen Temperatur')
plt.show()

In [ ]:
# Quantil-Quantil, Normalverteilung
from statsmodels.api import qqplot
fig=qqplot(Residuen, dist=norm)
fig.set_size_inches(8,6)
plt.title('QQ-Plot, Normalverteilung')
plt.show()

**Extremwerte**: Passt eine GEV-Verteilung?

In [ ]:
# Histogramm, GEV-Verteilung
fig,ax=plt.subplots()
fig.set_size_inches(8,6)

ax.hist(Residuen, bins=20, density=True, label='Residuen')
  
# Wahrscheinlichkeitsdichte normalverteilt
from scipy.stats import genextreme

c, loc, scl = genextreme.fit(Residuen) 
print(c, loc, scl)

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = genextreme.pdf(x, c, loc, scl)

ax.plot(x, p, 'k', linewidth=2, label='GEV-Verteilung')
plt.legend()
plt.title('Residuen Temperatur')
plt.show()

In [ ]:
# CDF, GEV-Verteilung
fig,ax=plt.subplots()
fig.set_size_inches(8,6)

res_cdf=ECDF(Residuen)
plt.plot(x,res_cdf(x),label='Residuen')
plt.plot(x,genextreme.cdf(x, c, loc, scl),label='GEV-Verteilung')
plt.legend()
plt.title('Residuen Temperatur')
plt.show()

In [ ]:
# QQ-Plot, GEV-Verteilung
c, loc, scl = genextreme.fit(Residuen)
fig=qqplot(Residuen, dist=genextreme, distargs=(c,), loc=loc, scale=scl)
fig.set_size_inches(8,6)
plt.title(f'QQ-Plot, GEV-Verteilung, c: {c:.3f}')
plt.show()

**Extremwerte**: Passt eine eine zensierte Verteilung besser ("Peak Over Threshold")?

In [ ]:
# QQ-Plot, Zensierte GEV-Verteilung
Res_filt=Residuen[Residuen>2]
c, loc, scl = genextreme.fit(Res_filt)
fig=qqplot(Res_filt, dist=genextreme, distargs=(c,), loc=loc, scale=scl)
fig.set_size_inches(8,6)
plt.title(f'QQ-Plot, GEV-Verteilung, c: {c:.3f}')
plt.show()

**GEV-Verteilung**: Formparameter

In [ ]:
# Vergleich Formparameter
x=np.linspace(-4, 4, 100)

c1, loc1, scl1 = 0.8, loc, scl
fig,ax=plt.subplots(figsize=(8,5.5))
ax.plot(x, genextreme.cdf(x, c, loc, scl), label=f'c: {c:.3f}, loc: {loc:.3f}, scl: {scl:.3f}',color='blue')
ax.plot(x, genextreme.cdf(x, c1, loc1, scl1), label=f'c: {c1:.3f}, loc: {loc1:.3f}, scl: {scl1:.3f}',color='red')
plt.title('GEV-Verteilungen, CDF')
plt.legend()
plt.show()
c

**Aufgaben Wiederkehrwert**:
- Was bedeuten die Zahlen unten?
- Für welche Wiederkehrperiode gelten sie?

In [ ]:
print(genextreme.ppf(0.01/12,c,loc,scl))
print(genextreme.ppf(1-0.01/12,c,loc,scl))

### Extremwertanalyse mit Libraryfunktionen

Hier ist die Variante mit Block Maxima gezeigt<p>
Die Library unterstützt auch Peak Over Threshold ('POT') von vorher

In [ ]:
# Benötigte Funktionen
from pyextremes import EVA, plot_mean_residual_life

In [ ]:
# Residuen müssen nicht gefiltert werden - statt den Werten (Residuen=pd.to_numeric(Luzern_Temp_Resid.resid).values) das Series Objekt Luzern_Temp_Resid.resid verwenden
Residuen_Reihe = Luzern_Temp_Resid.resid
Residuen_Reihe.head()

In [ ]:
# Modell trainieren mit der Blockmaxima-Methode
Modell_Max = EVA(Residuen_Reihe)
Modell_Max.get_extremes(method="BM", block_size="5Y", extremes_type="low")
Modell_Max.plot_extremes()
plt.show()

In [ ]:
# Nun können wir die Verteilung fitten
Modell_Max.fit_model(distribution='genextreme')
#Modell_Max
Modell_Max.distribution
# ... und anwenden ... hier Wiederkehrwert bei 120 Monaten
#Modell_Max.distribution.distribution.ppf(1-1/120)

In [ ]:
# Nützliche Zusammenfassung - ist das Modell gut?
Modell_Max.plot_diagnostic()
plt.show()

In [ ]:
# Wie überdurchschnittlich ist der Niederschlag eines Jahres- und Jahrzehntmonats?
Modell_Max.get_summary(return_period=[12, 120], alpha=0.95, n_samples=1000)

In [ ]:
# Tipps
# Verteilungsfamilie ändern
# Wie vorher: Verteilungsparameter festhalten
# Modell_Max.fit_model(distribution='genextreme',distribution_kwargs={"fc": 0.5})
# Schwelle bzw. Blockgrösse ändern

**Aufgaben Verteilungsfit**:
- Bestimmen Sie die Verteilung besonders kühler Monate
- Wie gross ist der Jahrzenteweiderkehrwert?
- Was heisst dieser Wert für einen Juli?